# Additional Calibration

This section calibrates additional parameters used for business cycle simulations and optimal stimulus calculations over the business cycle. 

Before we start, we import the libraries we need and the helper functions.

In [6]:
import pandas as pd
%run helpers.ipynb

We also read in the calibrated parameters:

In [7]:
params = pd.read_csv('output/params_suffstat.csv')
params = dict(params.values)
params

{'s': 0.035578947368421064,
 'u_bar': 0.05974999999999999,
 'x_bar': 0.557752047505985,
 'GC_bar': 0.19191729280673472,
 'M_bar': 0.5,
 'M_bar_l': 0.0,
 'M_bar_h': 2.0,
 'epsilon': 1.0,
 'epsilon_h': 1.5,
 'epsilon_l': 0.5,
 'eta': 0.6,
 'eta_l': 0.5,
 'eta_h': 0.7,
 'omega': 0.7071671173494025,
 'rho': 1.0808045754269548,
 'tau': 0.038731944011991065,
 'm_bar': 0.5100528390820994,
 'z0': 2.827830175507158,
 'z1': 2.2609135556386306,
 'GY_bar': 0.16101561237928394}

## Calibration of $\bar{Y}$

First, we calculate the average output. When calculating output, we use the following equation: 

                $Y(x, k) = \frac{f(x)}{s+f(x)}k = (1-u(x))\cdot k,$                [![Generic badge](https://img.shields.io/badge/MS19-Eq%201-purple?logo=read-the-docs)](https://www.pascalmichaillat.org/6.html)    

with the normalization $k = 1$.

In [8]:
k = 1
Y_bar = k*(1 - params['u_bar'])

## Calibration of $\gamma$

We now calibrate the preference parameter $\gamma$. When the Samuelson rule holds:

                $(g/c)^* = \frac{\gamma}{1-\gamma}.$                [![Generic badge](https://img.shields.io/badge/MS19-Eq%20A1-purple?logo=read-the-docs)](https://www.pascalmichaillat.org/6.html)    


In [9]:
gamma = 1/(1 + 1/params['GC_bar']) 

## Calibration of $r$

$r$ is the price rigidity.  At the efficient level, we have that

                $M = \frac{r}{\gamma r + (1-\gamma)\epsilon},$                [![Generic badge](https://img.shields.io/badge/MS19-Eq%20A4-purple?logo=read-the-docs)](https://www.pascalmichaillat.org/6.html)    


which implies that:

                $r = \frac{(1-\gamma)\epsilon M}{1 - \gamma M} = \frac{\overline{C/Y}\epsilon M}{1 - \overline{G/Y} M}.$                

In [10]:
CY_bar = CY_func(GC=params['GC_bar'])
r = (params['M_bar']*params['epsilon']*CY_bar)/(1 - params['M_bar']*params['GY_bar'])

## Calibration of $p_0$
We also need to calibrate the initial price level $p_0$ for the price mechanism $p(G)$. 

Looking at private demand, we have: 

                $\mathcal{U}_c = (1+\tau(x))p\alpha.$                [![Generic badge](https://img.shields.io/badge/MS19-Eq%2013-purple?logo=read-the-docs)](https://www.pascalmichaillat.org/6.html)    

Thus, with $\alpha = 1$, at the efficient level, we have that:

                $p_0 = \frac{\overline{\mathcal{U}_c}}{1+\bar\tau}.$

In [11]:
dUdc_bar = dUdc_func(gc=params['GC_bar'], gamma=gamma, **params)
p0 = dUdc_bar**r/(1 + params['tau']) 

## Storing Calibrated Parameters
We now store the newly calibrated parameters and proceed to business cycle simulations.
```{warning}
Only run the `to_csv` function when you are in **Binder** mode. If you run this under the **Live Code** option, saving to this directory will throw an error message. 
```

In [12]:
params = pd.DataFrame({'name':['k', 'Y_bar', 'gamma', 'CY_bar', 'r', 'p0'],
                       'value':[k, Y_bar, gamma, CY_bar, r, p0]})
params.to_csv('output/params_sim.csv', index=False)